<a href="https://colab.research.google.com/github/pinilDissanayaka/sinhala-gpt2-finetune/blob/main/sinhala_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [29]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import re

In [3]:
df = load_dataset("ihalage/sinhala-finetune-qa-eli5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.42k [00:00<?, ?B/s]

sinhala-finetune-qa-eli5.jsonl:   0%|          | 0.00/35.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [4]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")


print(f"Available Device : {device}")

Available Device : cuda


In [5]:
df

DatasetDict({
    train: Dataset({
        features: ['q_id', 'subreddit', 'url', 'sinhala_question', 'sinhala_answer', 'english_question', 'english_answer'],
        num_rows: 10000
    })
})

In [6]:
questions=list(df["train"]["sinhala_question"])

answers=list(df["train"]["sinhala_answer"])

In [33]:
def clean_text(list_text:list)->list:
  cleaned_text=list()

  for text in list_text:
   cleaned_text.append(re.sub(r'[a-zA-Z0-9]', "", text))

  return cleaned_text

In [34]:
questions=clean_text(questions)
answers=clean_text(answers)

In [35]:
from torch.utils.data import Dataset



class TextDataset(Dataset):
    def __init__(self, questions:list, answers:list, tokenizer):
        self.questions = questions
        self.answers = answers
        self.data=list()
        self.tokenizer = tokenizer

        for question, answer in tqdm(zip(self.questions, self.answers)):
            self.data.append("<startofstring> "+question+" <bot>:"+answer+" <endofstring>")

        self.data_tokenized = self.tokenizer(self.data, return_tensors="pt", padding=True, truncation=True)


        self.input_ids = self.data_tokenized["input_ids"]
        self.attention_mask = self.data_tokenized["attention_mask"]




    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return (self.input_ids[idx], self.attention_mask[idx])

In [36]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.add_special_tokens({"pad_token": "<pad>", "bos_token": "<startofstring>", "eos_token": "<endofstring>"})

tokenizer.add_tokens(["<bot>:"])



def train(model, optimizer, data, epochs):
  model=model.to(device)

  for epoch in tqdm(range(epochs)):
      for input_id, attention_mask in data:
        input_id = input_id.to(device)
        attention_mask = attention_mask.to(device)

        optimizer.zero_grad()
        loss = model(input_ids=input_id, attention_mask=attention_mask, labels=input_id).loss

        print(f"\n epoch: {epoch} loss: {loss}")

        loss.backward()
        optimizer.step()

  torch.save(model.state_dict(), f"model_epoch_{epochs}.pth")
  return model


def predict(model, text):
  text="<startofstring> "+text+" <bot>:"

  text_tokens=tokenizer(text)

  return model.generate(**text_tokens)

In [37]:
optimizer=Adam(params=model.parameters())

dataset=TextDataset(questions, answers, tokenizer)

data_loader=DataLoader(dataset, batch_size=2500, shuffle=True)

10000it [00:00, 634395.22it/s]


In [38]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [ ]:
model=train(model=model, optimizer=optimizer, data=dataset, epochs=3)

  0%|          | 0/3 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.

 epoch: 0 loss: 0.7558409571647644

 epoch: 0 loss: 0.6294891834259033

 epoch: 0 loss: 0.7412968277931213

 epoch: 0 loss: 0.6957498788833618

 epoch: 0 loss: 0.768056333065033

 epoch: 0 loss: 0.5926618576049805

 epoch: 0 loss: 0.6246228218078613

 epoch: 0 loss: 0.7112021446228027

 epoch: 0 loss: 0.7062342166900635

 epoch: 0 loss: 0.6799339652061462

 epoch: 0 loss: 0.5162594318389893

 epoch: 0 loss: 0.8005826473236084

 epoch: 0 loss: 0.21497869491577148

 epoch: 0 loss: 0.803976833820343

 epoch: 0 loss: 0.8956599235534668

 epoch: 0 loss: 0.753248393535614

 epoch: 0 loss: 0.8005456924438477

 epoch: 0 loss: 0.7684791684150696

 epoch: 0 loss: 0.7898289561271667

 epoch: 0 loss: 0.8103315830230713

 epoch: 0 loss: 0.4992547333240509

 epoch: 0 loss: 0.8138179183006287

 epoch: 0 loss: 0.8054177761077881

 epoch: 0 loss: 0.7915835976600647

 epoch: 0 loss: 0.3355100154876709

 epoch: 0 loss: 0.4114842116832733

 epoch: 0 loss